In [1]:
!pip install wptools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 28.2 MB/s eta 0:00:00
  Created wheel for html2text: filename=html2text-2024.2.26-py3-none-any.whl size=33111 sha256=d484b2fd4541d56616ac6a67ce6b91c9c00170f4eed8d64a5b8a03c7b3cacc0b
  Stored in directory: /root/.cache/pip/wheels/f3/96/6d/a7eba8f80d31cbd188a2787b81514d82fc5ae6943c44777659
Successfully built html2text


In [2]:
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 1.9 MB/s eta 0:00:00


In [3]:
!pip install beautifulsoup4

## Mount Google Drive

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
%cd 'drive/MyDrive/Thesis-MSc/wikipedia-scraping/'
# %cd 'Thesis-MSc/wikipedia-scraping/'

/content/drive/MyDrive/Thesis-MSc/wikipedia-scraping


In [6]:
import requests
import wptools
import re
import pandas as pd
import os
import csv
import json
from deep_translator import GoogleTranslator
import urllib3
from bs4 import BeautifulSoup
import string

In [7]:
page = wptools.page('Sabra_and_Shatila_massacre')
page.get_parse()

en.wikipedia.org (parse) Sabra_and_Shatila_massacre
en.wikipedia.org (imageinfo) File:Bodies of victims of the massac...
Sabra and Shatila massacre (en) data
{
  image: <list(1)> {'kind': 'parse-image', 'file': 'File:Bodies of...
  infobox: <dict(14)> title, partof, image, caption, location, nat...
  pageid: 52268
  parsetree: <str(128874)> <root><template><title>Short descriptio...
  requests: <list(2)> parse, imageinfo
  title: Sabra and Shatila massacre
  wikibase: Q208199
  wikidata_url: https://www.wikidata.org/wiki/Q208199
  wikitext: <str(87055)> {{Short description|1982 mass murder of c...
}


In [8]:
session = requests.Session()

url = "https://en.wikipedia.org/w/api.php"
params = {
    "action": "query",
    "format": "json",
    "titles": "Sabra_and_Shatila_massacre",
    "prop": "links",
    "pllimit": "max"
}

page = wptools.page('Sabra and Shatila massacre')

response = session.get(url=url, params=params)
data = response.json()
pages = data["query"]["pages"]

pg_count = 1
page_titles = []

print("Page %d" % pg_count)
for key, val in pages.items():
    for link in val["links"]:
        print(link["title"])
        page_titles.append(link["title"])

while "continue" in data:
    plcontinue = data["continue"]["plcontinue"]
    params["plcontinue"] = plcontinue

    response = session.get(url=url, params=params)
    data = response.json()
    pages = data["query"]["pages"]

    pg_count += 1

    print("\nPage %d" % pg_count)
    for key, val in pages.items():
        for link in val["links"]:
            print(link["title"])
            page_titles.append(link["title"])

print("%d titles found." % len(page_titles))

Page 1
1948 Palestinian expulsion from Lydda and Ramle
1956 Rafah massacre
1975 Beirut bus massacre
1978 South Lebanon conflict
1981 Iraqi embassy bombing in Beirut
1982 Beirut bombing
1982 Hama massacre
1982 Lebanon War
1982 kidnapping of Iranian diplomats
1983 Beirut barracks bombings
1983 US embassy bombing in Beirut
1984 Sohmor massacre
1984 United States embassy annex bombing in Beirut
1985 Beirut car bombings
2023 Israeli judicial reform protests
Abu Nidal
Abu Shusha massacre
Aharon Barak
Ahmad Tamal
Aishiyeh massacre
Al-Dawayima massacre
Al-Qaeda
Al Jazeera English
Alain Menargues
Amal Hamdan
Ami Popper
Amir Drori
Amnon Kapeliouk
Amos Yaron
Arab al-Mawasi massacre
Ari Folman
Ariel Sharon
Armenian Secret Army for the Liberation of Armenia
Assassination of Bachir Gemayel
Assassination of René Moawad
At Tiri incident
Avi Duda'i
Avraham Shalom
BBC
BBC Books
BBC News
Bachir Gemayel
Balad al-Shaykh massacre
Barnaby Mason
Battle of Kaukaba
Battle of Qnat
Battle of Sidon (1991)
Battle o

In [9]:
def get_arabic_name(english_phrase):
    # if we remove all numbers from a string and the string remains empty - then we won't translate that
    english_phrase_mod = ''.join([i for i in english_phrase if not i.isdigit()])
    english_phrase_mod = english_phrase_mod.translate(str.maketrans('', '', string.punctuation))
    english_phrase_mod = english_phrase_mod.strip()
    if english_phrase_mod == '':
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1
    else:
        english_phrase = english_phrase_mod

    if GoogleTranslator(source='auto', target='ar').translate(english_phrase) == english_phrase:
        print('Cannot translate phrase `{}` to arabic'.format(english_phrase))
        return -1, -1

    response = requests.get('http://en.wikipedia.org/wiki/{}'.format(english_phrase))
    if response.status_code != 200:
        print('Cannot find the link: {}'.format('http://en.wikipedia.org/wiki/{}'.format(english_phrase)))
        return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

    http = urllib3.PoolManager()
    url = 'http://en.wikipedia.org/wiki/{}'.format(english_phrase)
    response = http.request('GET', url)

    # get languages and links
    soup = BeautifulSoup(response.data)
    links = [(el.get('lang'), el.get('href')) for el in soup.select('li.interlanguage-link > a')]

    if links != []:
        for language, link in links:
            if language == 'ar':
                response = http.request('GET', link)
                soup = BeautifulSoup(response.data)

                arabic_phrase = soup.title.text
                arabic_phrase = re.sub('- ويكيبيديا', '', arabic_phrase)
                return arabic_phrase, link

    return GoogleTranslator(source='auto', target='ar').translate(english_phrase) + ' (مترجمه)', -1

def remove_file_instances(s):
    if '[[' in s and ']]' in s and 'File' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    elif '[[' in s and ']]' in s and 'Image' in s:
        ind2 = s.rfind(']')
        return s[ind2+1:]
    else:
        return s

# def to_delete(s):
#     return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s

def to_delete(s):
    # return 'name' in s or '{' in s or '}' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s
    return 'name' in s or '=' in s or 'class' in s or 'nowrap' in s or s in ['title', 'image', 'flag', 'Flagicon', 'flagicon', 'small'] or '.png' in s or '.svg' in s or 'File' in s or 'px' in s or 'Image' in s or '<small>' in s or 'Historical' in s or 'Current' in s or 'Formerly' in s or 'date' in s or len(s) == 1 or '\"' in s or '(' in s or 'not in' in s or 'Nowrap' in s or 'cite' in s or 'citation' in s or 'Cite' in s or 'http' in s

def get_list_items(s):
    # s = re.sub('{{.*?}}', '', s)
    # print(1, s)
    s = re.sub('\'', '', s)
#     print(2, s)
    s = re.sub(r'\[\[(?:[^\]|]*\|)?([^\]|]*)\]\]', r'\1', s)
#     print(3, s)
    s = re.sub('\n----', '<br>', s)
#     print(4, s)
    s = re.sub('\n\*', '<br>', s)
#     print(5, s)
    s = re.sub('<br />', '<br>', s)
#     print(6, s)
    s = re.sub('<br/>', '<br>', s)
    s = re.sub('<Br/>', '<br>', s)
#     print(7, s)
    s = re.sub('</ref>|ref||</ref>', '', s)
#     print(8, s)
    s = re.sub('</ref>', '', s)
#     print(9, s)
    s = re.sub('ref', '', s)
#     print(10, s)
#     s = re.sub('|', '', s)
    s = s.translate({ord('|'): '<br>'})
#     print(11, s)
    s = s.translate({ord('{'): None, ord('}'): None})
#     print(11, s)
#     if '<br>' in s:
    s = s.split("<br>")
#     print(12, s)
#     else:
#         s = s.split("\|")
    s = [e.strip() for e in s if e.strip() != '']
#     print(13, s)
    s = [remove_file_instances(e).strip() for e in s]
#     print(14, s)
    s = [e for e in s if not to_delete(e)]
#     print(15,  s)
    s = list(set(s)) # keep unique entries

    s = [e for e in s if not e.isdigit()] # remove entries that are just numbers
#     print('s is now: ', s)
    if len(s) > 1:
        new_s = []
        for e in s:
            if ',' in e:
                splitted_e = e.split(',')
                for ee in splitted_e:
                    new_s.append(ee)
            elif 'to' in e:
                splitted_e = e.split('to')
                for ee in splitted_e:
                    new_s.append(ee)
            else:
                new_s.append(e)
    #     print('s is now after: ', new_s)
        new_s = [e.strip() for e in new_s if e.strip() != '']
        return new_s
    else:
        return s


def jaccard(a, b):
    c = a.intersection(b)
    return float(len(c)) / (len(a) + len(b) - len(c))


In [10]:
def mkdir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

In [11]:
df_political_party_en = pd.DataFrame(columns=['Name', 'Ideology', 'Political Position', 'Founder(s)'])
df_political_party_ar = pd.DataFrame(columns=['الاسم','الإيديولوجيا', 'الموقف السياسي', 'المؤسس (المؤسسون)'])

In [12]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'ideology' in page.data['infobox']:
                    print(page)

                    leaders_types = [k for k in page.data['infobox'].keys() if 'leader' in k]
                    if leaders_types == []:
                        pass
                    else:
                        founders = ''
                        count = 0
                        for lt in leaders_types:
                            if count%2 == 0:
                                pass
#                                 founders += ''.join(get_list_items(page.data['infobox'][lt]))+ ':'
                            else:
                                print(page.data['infobox'][lt])
                                founders += ','.join(get_list_items(page.data['infobox'][lt])) + ','
                            count += 1

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    print(name)
                    if isinstance(name, list):
                        if len(name) > 1:
                            print(name)
                            jsims = []
                            for i, n in enumerate(name):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name = [name[jsims.index(max(jsims))]]
                            print('name after: ', name)
                    ideology = get_list_items(page.data['infobox']['ideology'])
                    position = get_list_items(page.data['infobox']['position']) if 'position' in page.data['infobox'] else '-'

                    df_political_party_en = pd.concat([df_political_party_en, pd.DataFrame({
                        'Name': [','.join(name if isinstance(name, list) else name)],
                        'Ideology': [','.join(ideology) if isinstance(ideology, list) else ideology],
                        'Political Position': [','.join(position) if isinstance(position, list) else position],
                        'Founder(s)': [founders],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('founders: ', founders)
                    print('ideology: ', ideology)
                    print('position: ', position)



                    # get the arabic name, ideology, political position, and founders of the political party
#                     page_ar = wptools.page(p, lang='ar')
#                     page_ar.get_parse()
#                     name_ar = page_ar.data['title']

                    if name != '':
                        print('START')
                        name_ar = []
                        for n in name:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('name: ', n)
                                print('name ar: ', n_ar),
                                print('name ar link: ', n_ar_link)
                                name_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        name_ar = '-'

                    print('START')
                    ideology_ar = []
                    for ideo in ideology:
                        ideo_ar, ideo_ar_link = get_arabic_name(ideo)
                        if ideo_ar != -1:
                            print('ideology: ', ideo)
                            print('ideology ar: ', ideo_ar),
                            print('ideo ar link: ', ideo_ar_link)
                            ideology_ar.append(ideo_ar)
                        else:
                            continue
                    print('END')

                    if position != '-':
                        position_ar = []
                        print('START')
                        for pos in position:
                            pos_ar, pos_ar_link = get_arabic_name(pos)
                            if pos_ar != -1:
                                print('position: ', pos)
                                print('position ar: ', pos_ar),
                                print('position ar link: ', pos_ar_link)
                                position_ar.append(pos_ar)
                            else:
                                continue
                        print('END')
                    else:
                        position_ar = '-'

                    print('START')
                    founders_ar = []
                    for f in founders.split(','):
                        if f.strip() != '':
                            f_ar, f_ar_link = get_arabic_name(f)
                            if f_ar != -1:
                                print('founder: ', f)
                                print('founder ar: ', f_ar),
                                print('founder ar link: ', f_ar_link)
                                founders_ar.append(f_ar)
                            else:
                                continue
                    print('END')


                    df_political_party_ar = pd.concat([df_political_party_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar)],
                        'الإيديولوجيا': [','.join(ideology_ar)],
                        'الموقف السياسي': [','.join(position_ar)],
                        'المؤسس (المؤسسون)': [','.join(founders_ar)],
                    })], ignore_index=True)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian expulsion from Lydda an...
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1975 Beirut bus massacre
en.wikipedia.org (imageinfo) File:Ain el Remmeneh Bus Massacre 1975.jpg
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.o

['Al-Qaeda']
-----------------
name:  ['Al-Qaeda']
founders:  
ideology:  ['Anti-communism', 'sfn', 'Sunni Islamism', 'Islamic fundamentalism', 'Factions:', 'Anti-Zionism', 'Collapsible list', 'loc', 'Anti-imperialism', 'Anti-Indian sentiment', '87 * Sunni–Shia alliance sfn', 'Willsky-Ciollo', 'Salafi jihadism', 'Jihadism', 'Antisemitism', 'harvnb', 'Factions: * Pan-Islamism sfn', 'Anti-Americanism', 'Deobandism', 'Sunni–Shia alliance', 'Salafism', 'Anti-LGBT', 'Muslim unity', '12 * Islamic fundamentalism * Anti-Americanism * Anti-communism * Anti-imperialism * Anti-Indian sentiment * Anti-LGBT * Antisemitism * Anti-Western imperialism * Anti-Zionism Plainlist', 'Anti-Western imperialism', 'Deobandi jihadism', 'Nbsp Plainlist', 'Gallagher', 'Wright', 'Sources:', 'Nbsp', 'Gunaratna', 'Bergen', 'Peter L.', 'Holy war', 'Inc.: Inside the Secret World of Osama bin Laden', 'New York: Free Press', '2001.', 'pp. 70–71', 'Wahhabism', 'Qutbism', 'Ahl-i Hadith', 'officially denied)', 'Introductio

en.wikipedia.org (parse) Al Jazeera English
en.wikipedia.org (parse) Alain Menargues
en.wikipedia.org (parse) Amal Hamdan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Ami Popper
en.wikipedia.org (parse) Amir Drori
en.wikipedia.org (imageinfo) File:אלוף פיקוד צפון אמיר דרורי.jpg
en.wikipedia.org (parse) Amnon Kapeliouk
en.wikipedia.org (parse) Amos Yaron
en.wikipedia.org (imageinfo) File:Amos Yaron.jpg
en.wikipedia.org (parse) Arab al-Mawasi massacre
en.wikipedia.org (parse) Ari Folman
en.wikipedia.org (imageinfo) File:Ari Folman during the publicati...
en.wikipedia.org (parse) Ariel Sharon
en.wikipedia.org (imageinfo) File:Ariel Sharon official portrait .

['hy', 'lang']
['hy', 'lang']
name before:  hy
name before:  lang
name after:  ['hy']
-----------------
name:  ['hy']
founders:  
ideology:  ['Left-wing nationalism', 'Revolutionary socialism', 'Marxism-Leninism', 'Armenian nationalism']
position:  ['Left-wing to far-left']
START
name:  hy
name ar:  هاى (مترجمه)
name ar link:  -1
END
START
Cannot find the link: http://en.wikipedia.org/wiki/Leftwing nationalism
ideology:  Left-wing nationalism
ideology ar:  القومية اليسارية (مترجمه)
ideo ar link:  -1
ideology:  Revolutionary socialism
ideology ar:  اشتراكية ثورية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D8%B4%D8%AA%D8%B1%D8%A7%D9%83%D9%8A%D8%A9_%D8%AB%D9%88%D8%B1%D9%8A%D8%A9
Cannot find the link: http://en.wikipedia.org/wiki/MarxismLeninism
ideology:  Marxism-Leninism
ideology ar:  الماركسيةاللينينية (مترجمه)
ideo ar link:  -1
ideology:  Armenian nationalism
ideology ar:  قومية أرمنية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%82%D9%88%D9%85%D9%8A%D8%A9_%D8%A3%D8%B1%D

en.wikipedia.org (parse) Assassination of Bachir Gemayel
en.wikipedia.org (imageinfo) File:The Kataeb Headquarters after E...
en.wikipedia.org (parse) Assassination of René Moawad
en.wikipedia.org (imageinfo) File:René Moawad.jpg
en.wikipedia.org (parse) At Tiri incident
en.wikipedia.org (parse) Avi Duda'i
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Avraham Shalom
en.wikipedia.org (parse) BBC
en.wikipedia.org (parse) BBC Books
en.wikipedia.org (imageinfo) File:BBCbooks.svg
en.wikipedia.org (parse) BBC News
en.wikipedia.org (parse) Bachir Gemayel
en.wikipedia.org (imageinfo) File:Bashir Gemayel (cropped).png
en.wikipedia.org (parse) Balad al-Shaykh massac

...

en.wikipedia.org (parse) South Lebanon security belt
en.wikipedia.org (parse) South Lebanon security belt administration
en.wikipedia.org (parse) St George's Church attack
en.wikipedia.org (parse) State University of New York Press
en.wikipedia.org (imageinfo) File:SUNYPressLogo.png
en.wikipedia.org (parse) State of Free Lebanon
en.wikipedia.org (parse) Syrian
en.wikipedia.org (parse) Syrian Social Nationalist Party


Rabie Banat
[[Antoun Saadeh]]
['Syrian Social Nationalist Party']
-----------------
name:  ['Syrian Social Nationalist Party']
founders:  Rabie Banat,Antoun Saadeh,
ideology:  ['Antisemitism', 'Economic populism', 'Anti-communism', 'Social nationalism', 'ubl', 'collapsible list', 'Syrian irredentism', 'Syrian nationalism', 'Fascism']
position:  ['and right', 'In the past', 'the party or elements of its ideology or membership have been erred to as belonging to both the political left', 'sometimes being labeled far-right.', 'Syncretic efn', 'and right', 'sometimes being labeled far-right.']
START
name:  Syrian Social Nationalist Party
name ar:  الحزب السوري القومي الاجتماعي 
name ar link:  https://ar.wikipedia.org/wiki/%D8%A7%D9%84%D8%AD%D8%B2%D8%A8_%D8%A7%D9%84%D8%B3%D9%88%D8%B1%D9%8A_%D8%A7%D9%84%D9%82%D9%88%D9%85%D9%8A_%D8%A7%D9%84%D8%A7%D8%AC%D8%AA%D9%85%D8%A7%D8%B9%D9%8A
END
START
ideology:  Antisemitism
ideology ar:  معاداة السامية 
ideo ar link:  https://ar.wikipedia.org/wiki/%D9%

en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (imageinfo) File:Syrian team with Milan.jpg
en.wikipedia.org (parse) Syrians in Lebanon
en.wikipedia.org (parse) Tantura massacre
en.wikipedia.org (imageinfo) File:Tantura, from Matson collection...
en.wikipedia.org (parse) Tel al-Zaatar massacre
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DestructionOfTe...
en.wikipedia.org (parse) The Daily Telegraph
en.wikipedia.org (imageinfo) File:The Daily Telegraph (British ne...
en.wikipedia.org (parse) The Dream (1987 film)
en.wikipedia.org (parse) The Guardian
en.wikipedia.org (imageinfo) File:The Guardian 28 May 2021.jpg
en.wikipedia.org (parse) The Independent
en.wikipedia.org (imageinfo) File:The Independent screenshot, 25 ...
en.wikipedia.org (parse) The Nation (U.S. periodical)
en.wikipedia.org (parse) The New Arab
en.wikipedia.org (parse) The New York Times
en.wikipedia.org (imageinfo) File:The New York Times, January 13,...
en.wikipedia.org (p

skipping Wikipedia:Citing sources
skipping Wikipedia:Protection policy
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite news
skipping Template:Cite web
skipping Template:Country data Israel
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:CS1 maint: unfit URL
skipping Category:Use Oxford spelling from October 2023
skipping Category:Use dmy dates from February 2024
skipping Category:Wikipedia articles needing page number citations from July 2024
skipping Category:Wikipedia articles needing page number citations from September 2022


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon
API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [13]:
df_political_party_en

,Name,Ideology,Political Position,Founder(s)
0,Al-Qaeda,"Anti-communism,sfn,Sunni Islamism,Islamic fund...",-,
1,hy,"Left-wing nationalism,Revolutionary socialism,...",Left-wing to far-left,
2,Islamic Resistance Movement,"sfn,Islamic fundamentalism,113 efn,Anti-Zionis...",-,"Yahya Sinwar,Khalil al-Hayya,Abu Omar Hassan,Y..."
3,Lebanese Forces,"Liberal conservatism,Lebanese nationalism,Chri...",Right-wing,"Samir Geagea,Bachir Gemayel,Vice-president,"
4,Lebanese Forces,"Anti-communism,Federalism in Lebanon,Lebanese ...",Right-wing to far-right,
5,Lebanese Front,"Anti-Arabism,Anti-communism,Factions:,Phoenici...",-,
6,Palestine Liberation Organization,"Pan-Arabism,Marxism,August 2024 * Factions:,Se...",Left-wing,"Mahmoud Abbas,"
7,Palestine Liberation Organization,"Pan-Arabism,Marxism,August 2024 * Factions:,Se...",Left-wing,"Mahmoud Abbas,"
8,Lebanese Kataeb Party,"Christian nationalism,Anti-communism,Social co...","Right-wing,far-right,Centre-right","Samy Gemayel,Pierre Gemayel,"
9,South Lebanon Army,"Anti-communism,Muslim-Christian Unity,Seculari...",-,","


In [14]:
df_political_party_ar

,الاسم,الإيديولوجيا,الموقف السياسي,المؤسس (المؤسسون)
0,القاعدة (مترجمه),"معاداة الشيوعية ,إس إف إن (مترجمه),إسلاموية ,أ...",-,
1,هاى (مترجمه),"القومية اليسارية (مترجمه),اشتراكية ثورية ,الما...",من اليسار إلى أقصى اليسار (مترجمه),
2,حركة حماس,"إس إف إن (مترجمه),أصولية إسلامية ,مادة (مترجمه...",-,"يحيى السنوار ,خليل الحية (مترجمه),ابو عمر حسن ..."
3,القوات اللبنانية,"محافظة ليبرالية ,قومية لبنانية ,ديمقراطية مسيحية",يمينية,"سمير جعجع ,بشير الجميل ,نائب الرئيس"
4,القوات اللبنانية,"معاداة الشيوعية ,الفيدرالية في لبنان (مترجمه),...",من اليمين إلى أقصى اليمين (مترجمه),
5,الجبهة اللبنانية,"معاداة العرب (مترجمه),معاداة الشيوعية ,الفصائل...",-,
6,منظمة التحرير الفلسطينية,"القومية العربية (مترجمه),ماركسية ,فصائل أغسطس ...",يسارية,
7,منظمة التحرير الفلسطينية,"القومية العربية (مترجمه),ماركسية ,فصائل أغسطس ...",يسارية,
8,حزب الكتائب اللبنانية (مترجمه),"قومية مسيحية ,معاداة الشيوعية ,محافظة اجتماعية...","يمينية ,أقصى اليمين (مترجمه),يمين الوسط (مترجمه)","سامي الجميل ,بيار الجميل"
9,جيش لبنان الجنوبي,"معاداة الشيوعية ,الوحدة الاسلامية المسيحية (مت...",-,


In [15]:
datasets_dir = 'datasets_updated/Sabra_and_Shatila_massacre/'

In [16]:
mkdir(datasets_dir)
df_political_party_en.to_excel(os.path.join(datasets_dir, 'political_parties_en.xlsx'), index=False)
df_political_party_ar.to_excel(os.path.join(datasets_dir, 'political_parties_ar.xlsx'), index=False)

In [17]:
df_politician_en = pd.DataFrame(columns=['Name', 'Political Party', 'Nationality', 'Religion'])
df_politician_ar = pd.DataFrame(columns=['الدين'       , 'الجنسيه'      , 'الحزب السياسي' , 'الاسم'])

In [18]:
for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'birth_date' in page.data['infobox']:
                    print(page)

                    name = get_list_items(page.data['infobox']['name']) if 'name' in page.data['infobox'] else '-'
                    native_name = get_list_items(page.data['infobox']['native_name']) if 'native_name' in page.data['infobox'] else '-'
                    nickname = get_list_items(page.data['infobox']['nickname']) if 'nickname' in page.data['infobox'] else '-'

                    religion = get_list_items(page.data['infobox']['religion']) if 'religion' in page.data['infobox'] else '-'
                    party = get_list_items(page.data['infobox']['party']) if 'party' in page.data['infobox'] else '-'
                    nationality = get_list_items(page.data['infobox']['nationality']) if 'nationality' in page.data['infobox'] else '-'

                    if name == '-':
                        name_inserted = native_name
                    else:
                        name_inserted = name
                    print(name_inserted)
                    if isinstance(name_inserted, list):
                        if len(name_inserted) > 1:
                            print(name_inserted)
                            jsims = []
                            for i, n in enumerate(name_inserted):
                                print('name before: ', n)
                                jacc = jaccard(set(n), set(p)) # compute jaccard similarity with the name and the page's title
                                jsims.append(jacc)
                                # name = [n]
                                #break
#                             name = list(name[0])
                            # the right name is the one that has highest jaccard sim score with the page's title
                            name_inserted = [name_inserted[jsims.index(max(jsims))]][0]
                            print('name after: ', name_inserted)
                        else:
                            if name_inserted != []:
                                print('name_inserted before: {}'.format(name_inserted))
                                # name_inserted = [name_inserted]
                                # name_inserted = ','.join(name_inserted) if isinstance(name_inserted, list) else name_inserted,
                                name_inserted = name_inserted[0]
                                print('name_inserted after: {}'.format(name_inserted))

                    df_politician_en = pd.concat([df_politician_en, pd.DataFrame({
                        'Name': [name_inserted],
                        'Political Party': [','.join(party) if isinstance(party, list) else party],
                        'Nationality': [','.join(nationality) if isinstance(nationality, list) else nationality],
                        'Religion': [','.join(religion) if isinstance(religion, list) else religion],
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('native_name: ', native_name)
                    print('nickname: ', nickname)
                    print('religion: ', religion)
                    print('party: ', party)
                    print('nationality: ', nationality)

                    # get name from wikipedia
                    # get political party from wikipedia
                    # translate nationality
                    # translate religion
                    name_ar, name_ar_link = get_arabic_name(p)
                    print('name ar: ', name_ar)

                    if party != '-':
                        print('START')
                        political_party_ar = []
                        for p in party:
                            p_ar, p_ar_link = get_arabic_name(p)
                            if p_ar != -1:
                                print('political party: ', p)
                                print('political party ar: ', p_ar),
                                print('political party ar link: ', p_ar_link)
                                political_party_ar.append(p_ar)
                            else:
                                continue
                        print('END')
                    else:
                        political_party_ar = '-'

                    if nationality != '-':
                        print('START')
                        nationality_ar = []
                        for n in nationality:
                            n_ar, n_ar_link = get_arabic_name(n)
                            if n_ar != -1:
                                print('nationality: ', n)
                                print('nationality ar: ', n_ar),
                                print('nationality ar link: ', n_ar_link)
                                nationality_ar.append(n_ar)
                            else:
                                continue
                        print('END')
                    else:
                        nationality_ar = '-'

                    if religion != '-':
                        print('START')
                        religion_ar = []
                        for r in religion:
                            r_ar, r_ar_link = get_arabic_name(r)
                            if r_ar != -1:
                                print('religion: ', r)
                                print('religion ar: ', r_ar),
                                print('religion ar link: ', r_ar_link)
                                religion_ar.append(r_ar)
                            else:
                                continue
                        print('END')
                    else:
                        religion_ar = '-'

                    df_politician_ar = pd.concat([df_politician_ar, pd.DataFrame({
                        'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                        'الحزب السياسي': [','.join(political_party_ar) if isinstance(political_party_ar, list) else political_party_ar],
                        'الجنسيه': [','.join(nationality_ar) if isinstance(nationality_ar, list) else nationality_ar],
                        'الدين': [','.join(religion_ar) if isinstance(religion_ar, list) else religion_ar],
                    })], ignore_index=True)

                    print('name ar: ', name_ar)
                    print('religion ar: ', political_party_ar)
                    print('party ar: ', party)
                    print('nationality ar: ', nationality_ar)


        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian expulsion from Lydda an...
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1975 Beirut bus massacre
en.wikipedia.org (imageinfo) File:Ain el Remmeneh Bus Massacre 1975.jpg
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.o

['Abu Nidal']
name_inserted before: ['Abu Nidal']
name_inserted after: Abu Nidal
-----------------
name:  ['Abu Nidal']
native_name:  ['أبو نضال', 'ar', 'lang']
nickname:  -
religion:  -
party:  []
nationality:  ['Palestinian']
name ar:  صبري البنا 
START
END
START
nationality:  Palestinian
nationality ar:  فلسطينيون 
nationality ar link:  https://ar.wikipedia.org/wiki/%D9%81%D9%84%D8%B3%D8%B7%D9%8A%D9%86%D9%8A%D9%88%D9%86
END
name ar:  صبري البنا 
religion ar:  []
party ar:  []
nationality ar:  ['فلسطينيون ']


en.wikipedia.org (parse) Abu Shusha massacre
en.wikipedia.org (imageinfo) File:גזר - גזר העתיקה - מקום מושבו ש...
en.wikipedia.org (parse) Aharon Barak
en.wikipedia.org (imageinfo) File:Aharon barak 01.jpg


['Aharon Barak']
name_inserted before: ['Aharon Barak']
name_inserted after: Aharon Barak
-----------------
name:  ['Aharon Barak']
native_name:  ['No bold', 'Script/Hebrew', 'אהרן ברק', 'אהרן ברק Script/Hebrew']
nickname:  -
religion:  -
party:  -
nationality:  ['Israeli']
name ar:  أهارون باراك 
START
nationality:  Israeli
nationality ar:  إسرائيلي (مترجمه)
nationality ar link:  -1
END
name ar:  أهارون باراك 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيلي (مترجمه)']


en.wikipedia.org (parse) Ahmad Tamal
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Aishiyeh massacre
en.wikipedia.org (parse) Al-Dawayima massacre
en.wikipedia.org (parse) Al-Qaeda
en.wikipedia.org (imageinfo) File:Flag of Jihad.svg
en.wikipedia.org (parse) Al Jazeera English
en.wikipedia.org (parse) Alain Menargues
en.wikipedia.org (parse) Amal Hamdan
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for not

...

en.wikipedia.org (parse) Yona Efrat
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Yossi Alpher
en.wikipedia.org (imageinfo) File:YossiAlpher.jpg


['Yossi Alpher']
name_inserted before: ['Yossi Alpher']
name_inserted after: Yossi Alpher
-----------------
name:  ['Yossi Alpher']
native_name:  -
nickname:  -
religion:  -
party:  -
nationality:  -
name ar:  يوسي ألفير (مترجمه)
name ar:  يوسي ألفير (مترجمه)
religion ar:  -
party ar:  -
nationality ar:  -


en.wikipedia.org (parse) Ze'ev Schiff
en.wikipedia.org (imageinfo) File:Zeev sh.jpeg


['Zeev Schiff']
name_inserted before: ['Zeev Schiff']
name_inserted after: Zeev Schiff
-----------------
name:  ['Zeev Schiff']
native_name:  ['זאב שיף']
nickname:  -
religion:  -
party:  -
nationality:  ['Israel']
name ar:  زئيف شيف 
START
nationality:  Israel
nationality ar:  إسرائيل 
nationality ar link:  https://ar.wikipedia.org/wiki/%D8%A5%D8%B3%D8%B1%D8%A7%D8%A6%D9%8A%D9%84
END
name ar:  زئيف شيف 
religion ar:  -
party ar:  -
nationality ar:  ['إسرائيل ']
skipping Wikipedia:Citing sources
skipping Wikipedia:Protection policy
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite news
skipping Template:Cite web
skipping Template:Country data Israel
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:CS1 maint: unfit URL
skipping Category:Use Oxford spelling from October 2023
skipping Category:Use dmy dates from 

en.wikipedia.org (parse) File:Lebanon adm location map.svg
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon
API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [19]:
df_politician_en

,Name,Political Party,Nationality,Religion
0,Abu Nidal,,Palestinian,-
1,Aharon Barak,-,Israeli,-
2,Ami Popper,-,-,-
3,Amir Drori,-,-,-
4,Amos Yaron,-,-,-
5,Ari Folman,-,-,-
6,nobold,plainlist,-,-
7,René Moawad,Independent,-,-
8,Avraham Shalom Bendor,-,Israel,-
9,Bachir Gemayel,Kataeb Party,-,-


In [20]:
df_politician_ar

,الدين,الجنسيه,الحزب السياسي,الاسم
0,-,فلسطينيون,,صبري البنا
1,-,إسرائيلي (مترجمه),-,أهارون باراك
2,-,-,-,أمي بوبر (مترجمه)
3,-,-,-,امير دروري (مترجمه)
4,-,-,-,عاموس يارون (مترجمه)
5,-,-,-,آري فولمان
6,-,-,قائمة عادية (مترجمه),أرئيل شارون
7,-,-,مستقل (توضيح),رينيه معوض
8,-,إسرائيل,-,إبراهيم بن دور
9,-,-,حزب الكتائب اللبنانية,بشير الجميل


In [21]:
mkdir(datasets_dir)
df_politician_en.to_excel(os.path.join(datasets_dir, 'politicians_en.xlsx'), index=False)
df_politician_ar.to_excel(os.path.join(datasets_dir, 'politicians_ar.xlsx'), index=False)

In [22]:
df_locations_en = pd.DataFrame(columns=['Name', 'Subdivision type', 'Subdivision name'])
df_locations_ar = pd.DataFrame(columns=['اسم التقسيم'         , 'نوع التقسيم' , 'الاسم'])

In [23]:
def isEnglish(s):
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

for p in page_titles:
    if 'Wikipedia' not in p and 'Template' not in p and 'Help' not in p and 'Category' not in p and 'Portal' not in p:
        page = wptools.page(p)
        try:
            page.get_parse(show=False)

            if page.data['infobox'] is not None:

                if 'government_type' in page.data['infobox'] or 'subdivision_type' in page.data['infobox']:
                    print(page)

                    if 'government_type' in page.data['infobox']:
                        name = get_list_items(page.data['infobox']['common_name']) if 'common_name' in page.data['infobox'] else  page.data['infobox']['conventional_long_name'] if 'conventional_long_name' in page.data['infobox'] else '-'
                    else:
                        name = get_list_items(page.data['infobox']['official_name']) if 'official_name' in page.data['infobox'] else page.data['infobox']['name'] if 'name' in  page.data['infobox'] else '-'

                    subdivision_type = get_list_items(page.data['infobox']['subdivision_type']) if 'subdivision_type' in page.data['infobox'] else '-'
                    subdivision_name = get_list_items(page.data['infobox']['subdivision_name']) if 'subdivision_name' in page.data['infobox'] else '-'

                    name = ','.join(name) if isinstance(name, list) and len(name) >= 1 else name
                    if isEnglish(name):
                        pass
                    else:
                        name = p

                    df_locations_en = pd.concat([df_locations_en, pd.DataFrame({
                        'Name': [name],
                        'Subdivision type': [','.join(subdivision_type) if isinstance(subdivision_type, list) else subdivision_type],
                        'Subdivision name': [','.join(subdivision_name) if isinstance(subdivision_name, list) else subdivision_name]
                    })], ignore_index=True)

                    print('-----------------')
                    print('name: ', name)
                    print('subdivision_type: ', subdivision_type)
                    print('subdivision_name: ', subdivision_name)


                    if isinstance(name, list):
                        name = name[0]
                    else:
                        pass

                    if name != '-':
                        name_ar, name_ar_link = get_arabic_name(name)
                        print('name ar: ', name_ar)

                        if subdivision_type != '-':
                            print('START')
                            subdivision_type_ar = []
                            for st in subdivision_type:
                                st_ar, st_ar_link = get_arabic_name(st)
                                if st_ar != -1:
                                    print('subdivision_type: ', st)
                                    print('subdivision_type ar: ', st_ar),
                                    print('subdivision_type ar link: ', st_ar_link)
                                    subdivision_type_ar.append(st_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_type_ar = '-'


                        if subdivision_name != '-':
                            print('START')
                            subdivision_name_ar = []
                            for sn in subdivision_name:
                                sn_ar, sn_ar_link = get_arabic_name(sn)
                                if sn_ar != -1:
                                    print('subdivision_name: ', sn)
                                    print('subdivision_name ar: ', sn_ar),
                                    print('subdivision_name ar link: ', sn_ar_link)
                                    subdivision_name_ar.append(sn_ar)
                                else:
                                    continue
                            print('END')
                        else:
                            subdivision_name_ar = '-'

                        df_locations_ar = pd.concat([df_locations_ar, pd.DataFrame({
                            'الاسم': [','.join(name_ar) if isinstance(name_ar, list) and len(name_ar) > 1 else name_ar],
                            'نوع التقسيم': [','.join(subdivision_type_ar) if isinstance(subdivision_type_ar, list) else subdivision_type_ar],
                            'اسم التقسيم': [','.join(subdivision_name_ar) if isinstance(subdivision_name_ar, list) else subdivision_name_ar],
                        })], ignore_index=True)

                    else:
                        print('skipping {} because name is not found'.format(p))

        except LookupError:
            continue


    else:
        print('skipping {}'.format(p))
        continue

en.wikipedia.org (parse) 1948 Palestinian expulsion from Lydda an...
en.wikipedia.org (parse) 1956 Rafah massacre
en.wikipedia.org (parse) 1975 Beirut bus massacre
en.wikipedia.org (imageinfo) File:Ain el Remmeneh Bus Massacre 1975.jpg
en.wikipedia.org (parse) 1978 South Lebanon conflict
en.wikipedia.org (imageinfo) File:Operation Litani V.jpg
en.wikipedia.org (parse) 1981 Iraqi embassy bombing in Beirut
en.wikipedia.org (parse) 1982 Beirut bombing
en.wikipedia.org (parse) 1982 Hama massacre
en.wikipedia.org (imageinfo) File:1982HamaMassacre.png
en.wikipedia.org (parse) 1982 Lebanon War
en.wikipedia.org (parse) 1982 kidnapping of Iranian diplomats
en.wikipedia.org (parse) 1983 Beirut barracks bombings
en.wikipedia.org (imageinfo) File:Beirutbarr.jpg
en.wikipedia.org (parse) 1983 US embassy bombing in Beirut
en.wikipedia.org (imageinfo) File:Beirutembassy.jpg
en.wikipedia.org (parse) 1984 Sohmor massacre
en.wikipedia.org (parse) 1984 United States embassy annex bombing...
en.wikipedia.o

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) Benny Morris
en.wikipedia.org (imageinfo) File:Benny morris.jpg
en.wikipedia.org (parse) Bernard Lewis
en.wikipedia.org (imageinfo) File:Bernard Lewis in 2012 (1).JPG
en.wikipedia.org (parse) Black Saturday (Lebanon)
en.wikipedia.org (parse) Black Thursday (Lebanon)
en.wikipedia.org (parse) Bombing of Lebanon (June 1981)
en.wikipedia.org (parse) Bourj Hammoud massacre
en.wikipedia.org (parse) Brandeis University
en.wikipedia.org (imageinfo) File:Brandeis University seal.svg
en.wikipedia.org (parse) Bus massacre
en.wikipedia.org (imageinfo) File:Ain el Remmeneh Bus Massacre 1975.jpg
en.wikipedia.org (parse) Cambridge University Press
en.wikipedia.org (imageinfo) File:Cambridge University Press logo.svg
en.wikipedia.org (parse) Castration
en.wikipedia.org (parse) Cave of the Patriarchs massacre
en.wikipedia.org (imageinfo) File:Palestine Hebron Cave of the Pa...
en.wikipedia.org (parse) Chekka massacre
en.wikipedia.org (parse) Chouf massacres
en.wikipedia.org (pa

-----------------
name:  Iraq
subdivision_type:  -
subdivision_name:  -
name ar:  العراق 


en.wikipedia.org (parse) Islam in Lebanon
en.wikipedia.org (imageinfo) File:Mohammad Al-Amin Mosque during ...
en.wikipedia.org (parse) Israel's Secret Wars: A History of Israe...
en.wikipedia.org (imageinfo) File:Israels Secret Wars.JPG
en.wikipedia.org (parse) Israel Defense Forces
en.wikipedia.org (parse) Israel–Hamas war hostage crisis
en.wikipedia.org (imageinfo) File:Iron Swords 141023 Kirya Bring ...
en.wikipedia.org (parse) Ithaca Press
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) JSTOR (identifier)
en.wikipedia.org (parse) Janet Lee Stevens
en.wikipedia.org (parse) January 1986 Lebanese Forces coup
en.wikipedia.org (parse) Jeeps
en.wikipedia.org 

...

en.wikipedia.org (parse) Syrian
en.wikipedia.org (parse) Syrian Social Nationalist Party
en.wikipedia.org (parse) Syrian intervention in the Lebanese Civil War
en.wikipedia.org (imageinfo) File:Syrian team with Milan.jpg
en.wikipedia.org (parse) Syrians in Lebanon
en.wikipedia.org (parse) Tantura massacre
en.wikipedia.org (imageinfo) File:Tantura, from Matson collection...
en.wikipedia.org (parse) Tel al-Zaatar massacre
en.wikipedia.org (imageinfo) File:JeanJacquesKurz-DestructionOfTe...
en.wikipedia.org (parse) The Daily Telegraph
en.wikipedia.org (imageinfo) File:The Daily Telegraph (British ne...
en.wikipedia.org (parse) The Dream (1987 film)
en.wikipedia.org (parse) The Guardian
en.wikipedia.org (imageinfo) File:The Guardian 28 May 2021.jpg
en.wikipedia.org (parse) The Independent
en.wikipedia.org (imageinfo) File:The Independent screenshot, 25 ...
en.wikipedia.org (parse) The Nation (U.S. periodical)
en.wikipedia.org (parse) The New Arab
en.wikipedia.org (parse) The New York Times

-----------------
name:  Beirut
subdivision_type:  ['Country']
subdivision_name:  ['Lebanon']
name ar:  بيروت 
START
subdivision_type:  Country
subdivision_type ar:  بلد 
subdivision_type ar link:  https://ar.wikipedia.org/wiki/%D8%A8%D9%84%D8%AF
END
START
subdivision_name:  Lebanon
subdivision_name ar:  لبنان 
subdivision_name ar link:  https://ar.wikipedia.org/wiki/%D9%84%D8%A8%D9%86%D8%A7%D9%86
END


en.wikipedia.org (parse) William B. Quandt
en.wikipedia.org (imageinfo) File:William B. Quandt.jpg
en.wikipedia.org (parse) William Schabas
en.wikipedia.org (imageinfo) File:ICTY Legacy Symposium Panel III...
en.wikipedia.org (parse) World Press Review
en.wikipedia.org (parse) Yehoshua Saguy
en.wikipedia.org (imageinfo) File:Yehoshua Sagi, February 1979 (D...
en.wikipedia.org (parse) Yitzhak Kahan
en.wikipedia.org (parse) Yitzhak Shamir
en.wikipedia.org (imageinfo) File:Yitzhak Shamir 1992 Dan Hadani ...
en.wikipedia.org (parse) Yona Efrat
API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Yossi Alpher
en.wikipedia.org (imageinfo) File:YossiAlpher.jpg
en.wikipe

skipping Wikipedia:Citing sources
skipping Wikipedia:Protection policy
skipping Template:Campaignbox Lebanese Civil War
skipping Template:Cite news
skipping Template:Cite web
skipping Template:Country data Israel
skipping Template:Massacres against Palestinians
skipping Template talk:Campaignbox Lebanese Civil War
skipping Template talk:Massacres against Palestinians
skipping Help:Authority control
skipping Category:CS1 maint: unfit URL
skipping Category:Use Oxford spelling from October 2023
skipping Category:Use dmy dates from February 2024
skipping Category:Wikipedia articles needing page number citations from July 2024
skipping Category:Wikipedia articles needing page number citations from September 2022


API error: {'code': 'missingtitle', 'info': "The page you specified doesn't exist.", 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}
en.wikipedia.org (parse) Module:Location map/data/Lebanon
API error: {'code': 'notwikitext', 'info': '"prop=parsetree" is only supported for wikitext content.', 'docref': 'See https://en.wikipedia.org/w/api.php for API usage. Subscribe to the mediawiki-api-announce mailing list at &lt;https://lists.wikimedia.org/postorius/lists/mediawiki-api-announce.lists.wikimedia.org/&gt; for notice of API deprecations and breaking changes.'}


In [24]:
df_locations_en

,Name,Subdivision type,Subdivision name
0,Beirut,Country,Lebanon
1,Iraq,-,-
2,Lebanon,-,-
3,Nahariyya,Country,ISR
4,Sabra,Country,Lebanon
5,Saliha,Geopolitical entity,Mandatory Palestine
6,Singapore,-,-
7,Israeli occupation of Southern Lebanon,-,-
8,Southern Lebanon,-,-
9,Beirut,Country,Lebanon


In [25]:
df_locations_ar

,اسم التقسيم,نوع التقسيم,الاسم
0,لبنان,بلد,بيروت
1,-,-,العراق
2,-,-,لبنان
3,الاستخبارات والمراقبة والرصد (مترجمه),بلد,نهاريا
4,لبنان,بلد,صبرة (مترجمه)
5,الانتداب البريطاني على فلسطين,علم الوجود الجغرافي السياسي,صلحا
6,-,-,سنغافورة
7,-,-,الاحتلال الإسرائيلي لجنوب لبنان
8,-,-,جنوب لبنان
9,لبنان,بلد,بيروت


In [26]:
mkdir(datasets_dir)
df_locations_en.to_excel(os.path.join(datasets_dir, 'locations_en.xlsx'), index=False)
df_locations_ar.to_excel(os.path.join(datasets_dir, 'locations_ar.xlsx'), index=False)